In [90]:
import pandas as pd
import pandasql as pdsql

In [91]:
filename = 'weather_underground.csv'
weather_data = pd.read_csv('./data/' + filename)

In [92]:
weather_data.describe()

,maxpressurem,maxdewptm,maxpressurei,maxdewpti,since1julheatingdegreedaysnormal,heatingdegreedaysnormal,since1sepcoolingdegreedaysnormal,hail,since1julsnowfallm,since1julheatingdegreedays,...,precipi,snowfalli,since1jancoolingdegreedaysnormal,precipm,snowfallm,thunder,monthtodateheatingdegreedays,meantempi,maxvism,meantempm
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,0.0,30.0,3.000000e+01,30.000000,...,30.000000,30.0,30.000000,30.000000,30.0,30.0,30.000000,30.000000,30.000000,30.000000
mean,1016.733333,14.066667,30.031333,57.266667,4717.700000,4.166667,NaN,0.0,1.572300e+02,4691.100000,...,0.172667,0.0,34.300000,4.329000,0.0,0.0,61.100000,64.300000,15.733333,17.900000
std,4.425111,4.940566,0.127840,8.920852,35.338901,1.821014,NaN,0.0,2.890759e-14,28.807746,...,0.437051,0.0,17.186904,11.119453,0.0,0.0,28.807746,6.685806,1.142693,3.744882
min,1007.000000,4.000000,29.740000,39.000000,4646.000000,2.000000,NaN,0.0,1.572300e+02,4635.000000,...,0.000000,0.0,13.000000,0.000000,0.0,0.0,5.000000,55.000000,10.000000,13.000000
25%,1014.250000,10.000000,29.965000,50.000000,4692.250000,3.000000,NaN,0.0,1.572300e+02,4669.500000,...,0.000000,0.0,20.250000,0.000000,0.0,0.0,39.500000,60.250000,16.000000,16.000000
50%,1017.000000,14.500000,30.030000,58.000000,4725.000000,4.000000,NaN,0.0,1.572300e+02,4690.500000,...,0.000000,0.0,30.000000,0.000000,0.0,0.0,60.500000,63.000000,16.000000,17.000000
75%,1019.000000,17.750000,30.097500,63.750000,4748.250000,5.750000,NaN,0.0,1.572300e+02,4723.000000,...,0.092500,0.0,46.250000,2.350000,0.0,0.0,93.000000,67.750000,16.000000,19.750000
max,1026.000000,21.000000,30.310000,70.000000,4763.000000,8.000000,NaN,0.0,1.572300e+02,4724.000000,...,2.180000,0.0,71.000000,55.370000,0.0,0.0,94.000000,78.000000,16.000000,26.000000


#### Q1: Find the number of days when it rained

In [98]:
def num_rainy_days(weather_data):
    
    q = """
    SELECT count(*)
    FROM weather_data wd
    WHERE wd.rain == 1
    """
    
    rainy_days = pdsql.sqldf(q.lower(), locals())
    
    return rainy_days

In [99]:
num_rainy_days(weather_data)

,count(*)
0,10


#### Q2: The SQL query should return two columns and two rows - whether it was foggy or not (0 or 1) and the max maxtempi for that fog value (i.e., the maximum max temperature for both foggy and non-foggy days).

In [114]:
def max_temp_aggregate_by_fog(weather_data):
    
    q = """
    SELECT wd.fog, cast(max(maxtempi) as integer)
    FROM weather_data wd
    GROUP BY wd.fog
    """
    
    foggy_days = pdsql.sqldf(q.lower(), locals())
                       
    return foggy_days

In [115]:
max_temp_aggregate_by_fog(weather_data)

,fog,cast(max(maxtempi) as integer)
0,0,86
1,1,81
